In [ ]:
import pandas as pd

import gc
####

import seaborn as sns
import plotly.express as px

import numpy as np

import sqlite3
from sqlite3 import Error
import os
import re, string
from nltk.corpus import stopwords
import nltk
import matplotlib.pyplot as plt

import torch
#from transformers import AutoTokenizer, BertForSequenceClassification
## felipe


from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
from sklearn.decomposition import PCA

nltk.download("stopwords")
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')


import pandas as pd
import re, string


pd.options.display.max_columns = None

#from sentence_transformers import SentenceTransformer, util
#from transformers import *

import spacy

nlp = spacy.load('es_core_news_lg', disable=['tagger', 'parser', 'ner'])



from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn import preprocessing

In [ ]:
!python -m spacy download es_core_news_lg

In [ ]:
!python -m spacy download es_core_news_lg

%pip install pycorrector
%pip install unidecode
%pip install pyspellchecker

# etl

In [ ]:
import pandas as pd
import sqlite3

In [ ]:


query1 = "SELECT NUMERO_PQR,DESCRIPCION_PQR FROM pqr_priorizar where MEDIO_RECEPCION !=  'VIRTUAL' and unico=1 and SINTOMA_PQR ='CONCEPTOS FACTURADOS' and FECHA_CREACION >= '2023-07-17' and TECNOLOGIA ='FO'"

query2 = "(SELECT * FROM altas where ST_TIPO_SERVICIO='FTTH' AND FASE='ORDERF' AND (tipoTramite= 'VTA' OR tipoTramite=  'Sales')) AS myquery"

In [ ]:

# Connect to the SQLite database file
conn = sqlite3.connect('/content/on2/pqr_priorizar.db')

# Read the SQL query into a pandas DataFrame

pqrs = pd.read_sql_query(query1, conn)

# Close the database connection
conn.close()

# Print the DataFrame
pqrs

#MDM-PQR-39191683

# TRASNFORMAR LAS DATOS DE TEXTO

In [ ]:
sys.path.append('../librerias')

In [ ]:
from clean_text import clean_text

In [16]:
pqrs = pqrs.fillna("vacio")



In [17]:
pqrs.isnull().sum()

NUMERO_PQR         0
DESCRIPCION_PQR    0
dtype: int64

In [21]:
#declare answers and questions
descripcion=pqrs.DESCRIPCION_PQR





#clean_descripcion=[k for k in clean_descripcion if 'soporte'  in k]
clean_descripcion = [clean_text(des) for des in descripcion]


len(clean_descripcion)

1928

In [22]:
clean_descripcion[0]

'nombre nohora beatriz gacha rodriguez   cc    direccion   b a   asunto informacion factura'

spelling

In [23]:
from tqdm import tqdm
from spelling import correct_sentence


hola


In [24]:
corrected_sentences = []
for text in tqdm(clean_descripcion, desc="Processing"):
    corrected_sentence = correct_sentence(text)
    corrected_sentences.append(corrected_sentence)

clean_descripcion=corrected_sentences

Processing: 100%|██████████| 1928/1928 [05:01<00:00,  6.40it/s]


lemma token stemm

In [ ]:
from lemastem import data_preprocessing

In [26]:

clean_descripcion=data_preprocessing(clean_descripcion)
clean_descripcion[190]

['aldar',
 'asunto',
 'solicitar',
 'cancelacion',
 'serio',
 'adicional',
 'hbo',
 'paramount',
 'adema',
 'solicitar',
 'envio',
 'factura',
 'inactivo']

In [27]:
clean_descripcion[0]

['ahora', 'rodrigues', 'asunto', 'factura']

In [28]:
merged_data=[]
for i in range(len(clean_descripcion)):
    merged_data.append(" ".join(clean_descripcion[i]))

merged_df=pd.DataFrame(merged_data,columns =['descripcion'])


In [29]:
from unidecode import unidecode

def remove_accents(text):
    return unidecode(text)

# Aplicar la función a la columna 'Texto'
merged_df['descripcion'] = merged_df['descripcion'].apply(remove_accents)

In [30]:
merged_df

,descripcion
0,ahora rodrigues asunto factura
1,incremento factura
2,preparar solicitar aclaracion factura
3,senoro informar desear saber si s estar cobrar...
4,solicitar factura
...,...
1923,compensacion may
1924,encontrar masivo factura llego normal sin ajus...
1925,no aplicar descuento
1926,factura envio factura



clean_descripcion = [text for text in clean_descripcion if "svas" not in text and "directv" not in text  and "winsport" not in text and "hbo" not in text  ]

len(clean_descripcion)

In [32]:
from tfidf import TFIDF
import joblib

In [34]:
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from joblib import dump


X = merged_df['descripcion']
X_test=merged_df['descripcion']


X_train,_ = TFIDF(X,X_test)

# Create a new instance of PCA with the optimal number of components

pca = joblib.load('/content/PRIORIZAR/modelo/pca_binario.joblib')

#pca = PCA(n_components=585)


X_train_new = pca.fit_transform(X_train)


print("train with old features: ",np.array(X).shape)
print("train with new features:" ,np.array(X_train_new).shape)



tf-idf with 1117 features
train with old features:  (1928,)
train with new features: (1928, 804)


# cargar modelo

In [35]:
from joblib import dump

import joblib


model = joblib.load('/content/PRIORIZAR/modelo/modelo_binario.joblib')

print(model)

LGBMClassifier(bagging_fraction=0.9747337180903012, bagging_freq=4,
               feature_fraction=0.6526645750030313,
               lambda_l1=0.02873208934418634, lambda_l2=2.3446621696223424,
               learning_rate=0.0815126268370284, min_child_samples=35,
               n_estimators=88, num_leaves=35)


In [36]:
y_pred = model.predict(X_train_new)

pqrs['prediccion_algoritmo'] = y_pred

pqrs['descripcion_clean']=merged_df['descripcion']



# Lista de palabras a filtrar
words_to_remove = ['svas', 'directv', 'paramont', 'hbo']

# Filtrar las filas del DataFrame
c_facturados_correcion = pqrs[~pqrs['descripcion_clean'].str.contains('|'.join(words_to_remove))]

c_facturados_SVAS = pqrs[pqrs['descripcion_clean'].str.contains('|'.join(words_to_remove))]



c_facturados_correcion


,NUMERO_PQR,DESCRIPCION_PQR,prediccion_algoritmo,descripcion_clean
0,MDM-PQR-39939149,Nombre/ NOHORA BEATRIZ GACHA RODRIGUEZ Cc/ 1...,0.0,ahora rodrigues asunto factura
1,MDM-PQR-39939358,Cliente NELSON JAMPIERRE RODRÍGUEZ GARCIA Cédu...,1.0,incremento factura
2,MDM-PQR-39939410,karepara WhatsApp solicita aclaración de factura,1.0,preparar solicitar aclaracion factura
3,MDM-PQR-39939507,NOMBRE: PILAR INDUSTRIAS METALICAS CRUZ M.R. S...,1.0,senoro informar desear saber si s estar cobrar...
4,MDM-PQR-39939580,Línea: 6017152316 Nombre de quien se comuni...,1.0,solicitar factura
...,...,...,...,...
1923,MDM-PQR-40062599,whatsapp// cliente desea compensación falla Ma...,0.0,compensacion may
1924,MDM-PQR-40062700,Nombre: MANUEL JOSE GONZALEZ ROBLED cc: 102655...,1.0,encontrar masivo factura llego normal sin ajus...
1925,MDM-PQR-40062840,CLIENTE INDICA QUE NO LE APLICARON UN DESCUENT...,1.0,no aplicar descuento
1926,MDM-PQR-40062863,whatsapp// validar motivo factura $0 / envío d...,1.0,factura envio factura


In [37]:
from datetime import date

today = date.today()

year, week_num, day_of_week = today.isocalendar()





In [53]:
errores = len(c_facturados_correcion[c_facturados_correcion['prediccion_algoritmo'] == 0])
total  =len(c_facturados_correcion)
sva=len(c_facturados_SVAS)
error=(errores/total)*100
semana=week_num-1
index='-'.join(map(str, [year,semana]))



data = {'level_0':0,
        'errores': [errores],
        'total': [total],
        'sva': [sva],
        '%_error':[error],
        'semana':[semana],
        'index':[index]}

tracking_tipificacion = pd.DataFrame(data)


#tracking_c_facturado

In [54]:
file_path = "/content/PRIORIZAR/data/precision.txt"
with open(file_path, "r") as file:
    precision = file.read()


tracking_tipificacion["accuracy"]=precision

tracking_tipificacion

,level_0,errores,total,sva,%_error,semana,index,accuracy
0,0,593,1895,33,31.292876,29,2023-29,0.8


In [55]:
tracking_tipificacion.columns

Index(['level_0', 'errores', 'total', 'sva', '%_error', 'semana', 'index',
       'accuracy'],
      dtype='object')

cargando data apra tablero

In [56]:

 
# append data frame to CSV file
tracking_tipificacion.to_csv('/content/tracking_c_facturado.csv', mode='a', index=False, header=False)
pd.read_csv('/content/tracking_c_facturado.csv')

#!rclone copy  /content/tracking_c_facturado.csv one:/analitica_datos/tracking_c_facturado

In [59]:
!rclone copy  /content/tracking_c_facturado.csv one:/analitica_datos/tracking_c_facturado

In [60]:
import pandas as pd
from datetime import date


from datetime import datetime

# Obtener la fecha y hora actual
current_date = datetime.now()
# Get the current date
# Specify the root directory path
root_path = '/content/PRIORIZAR/SALIDAS/'

# Export the DataFrame to a CSV file with the root path and current date in the filename
filename1 = f"{root_path}c_facturados_correcion_{current_date}.csv"
filename2 = f"{root_path}c_facturados_SVAS_{current_date}.csv"


c_facturados_correcion.to_csv(filename1, index=False)

c_facturados_SVAS.to_csv(filename2, index=False)




#pandas_c_F.to_csv('c_facturados.csv', index=False)

print("DONE")

DONE


# modelo retipificacion

filter data with 0 label

In [61]:
retipificacion = c_facturados_correcion[c_facturados_correcion['prediccion_algoritmo'] == 0.0]
retipificacion

,NUMERO_PQR,DESCRIPCION_PQR,prediccion_algoritmo,descripcion_clean
0,MDM-PQR-39939149,Nombre/ NOHORA BEATRIZ GACHA RODRIGUEZ Cc/ 1...,0.0,ahora rodrigues asunto factura
6,MDM-PQR-39939719,WHATSAPP// Cliente se comunica para actualizar...,0.0,actualizar mail envio copia factura
7,MDM-PQR-39939847,Linea: 6012253776 Cuenta: 2368998 Nombre: CHR...,0.0,aumento factura solicitar verificar motivo
9,MDM-PQR-39940029,Cliente Solicita aclaración de aplicación de p...,0.0,solicitar aclaracion aplicacion pago ejecutado...
17,MDM-PQR-39940955,LINEA:6013118150 NOMBRE:LILIA TORRES FORERO ...,0.0,no llegar factura
...,...,...,...,...
1906,MDM-PQR-40061499,Cliente OSCAR DANILO RAMIREZ Cédula de Ciudada...,0.0,variar factura
1911,MDM-PQR-40061722,Cliente YULIANA CATERIN GALINDO AVILA Cédula d...,0.0,catering lindo avisar contar facturacion valid...
1914,MDM-PQR-40061926,LINEA: 6013614881 NOMBRE: Pilar Moreno ...,0.0,factura llego pago pasado
1920,MDM-PQR-40062400,CLIENTE INDICA QUE SE LE AUMENTO LA FACTURA,0.0,aumento factura


In [62]:
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split



X = retipificacion['descripcion_clean']
X_test=retipificacion['descripcion_clean']


X_train,_ = TFIDF(X,X_test)

# Create a new instance of PCA with the optimal number of components

pca = joblib.load('/content/PRIORIZAR/modelo/pca_retipificacion.joblib')

#pca = PCA(n_components=361)


X_train_new = pca.fit_transform(X_train)


print("train with old features: ",np.array(X).shape)
print("train with new features:" ,np.array(X_train_new).shape)



tf-idf with 646 features
train with old features:  (593,)
train with new features: (593, 249)


In [63]:
model = joblib.load('/content/PRIORIZAR/modelo/retipificacion.joblib')

print(model)

LGBMClassifier(bagging_fraction=0.8902288478379407, bagging_freq=1,
               feature_fraction=0.9963731191377936,
               lambda_l1=0.31449684221997787, lambda_l2=0.028561307947605728,
               learning_rate=0.0988198318878283, min_child_samples=43,
               n_estimators=82, num_leaves=46)


In [64]:
y_pred = model.predict(X_train_new)

retipificacion["retipificacion_algoritmo"]=y_pred

<ipython-input-64-9505f33dc576>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  retipificacion["retipificacion_algoritmo"]=y_pred


In [65]:
mapping = {0: 'BENEFICIO DE RETENCION', 1: 'INCREMENTOS DE FACTURA', 2: 'BAJA NO INGRESADA',  3: 'CANCELACION',4: 'COMPENSACION'}

# Separate features and target variable

retipificacion['tipificacion']= retipificacion['retipificacion_algoritmo'].replace(mapping)

retipificacion['index']=index


<ipython-input-65-660200b608e7>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  retipificacion['tipificacion']= retipificacion['retipificacion_algoritmo'].replace(mapping)
<ipython-input-65-660200b608e7>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  retipificacion['index']=index


In [66]:
file_path = "/content/PRIORIZAR/data/f1scoreretipificacion.txt"
df = pd.read_csv(file_path, sep=',', header=None)
f1 = df.values.flatten()

retipificacion.loc[retipificacion["retipificacion_algoritmo"] == 0, "f1-score"] = f1[0]
retipificacion.loc[retipificacion["retipificacion_algoritmo"] == 1, "f1-score"] = f1[1]
retipificacion.loc[retipificacion["retipificacion_algoritmo"] == 2, "f1-score"] = f1[2]
retipificacion.loc[retipificacion["retipificacion_algoritmo"] == 3, "f1-score"] = f1[3]
retipificacion.loc[retipificacion["retipificacion_algoritmo"] == 4, "f1-score"] = f1[4]


retipificacion

<ipython-input-66-273398139352>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  retipificacion.loc[retipificacion["retipificacion_algoritmo"] == 0, "f1-score"] = f1[0]


,NUMERO_PQR,DESCRIPCION_PQR,prediccion_algoritmo,descripcion_clean,retipificacion_algoritmo,tipificacion,index,f1-score
0,MDM-PQR-39939149,Nombre/ NOHORA BEATRIZ GACHA RODRIGUEZ Cc/ 1...,0.0,ahora rodrigues asunto factura,4,COMPENSACION,2023-29,0.93
6,MDM-PQR-39939719,WHATSAPP// Cliente se comunica para actualizar...,0.0,actualizar mail envio copia factura,4,COMPENSACION,2023-29,0.93
7,MDM-PQR-39939847,Linea: 6012253776 Cuenta: 2368998 Nombre: CHR...,0.0,aumento factura solicitar verificar motivo,4,COMPENSACION,2023-29,0.93
9,MDM-PQR-39940029,Cliente Solicita aclaración de aplicación de p...,0.0,solicitar aclaracion aplicacion pago ejecutado...,4,COMPENSACION,2023-29,0.93
17,MDM-PQR-39940955,LINEA:6013118150 NOMBRE:LILIA TORRES FORERO ...,0.0,no llegar factura,1,INCREMENTOS DE FACTURA,2023-29,0.93
...,...,...,...,...,...,...,...,...
1906,MDM-PQR-40061499,Cliente OSCAR DANILO RAMIREZ Cédula de Ciudada...,0.0,variar factura,2,BAJA NO INGRESADA,2023-29,0.90
1911,MDM-PQR-40061722,Cliente YULIANA CATERIN GALINDO AVILA Cédula d...,0.0,catering lindo avisar contar facturacion valid...,4,COMPENSACION,2023-29,0.93
1914,MDM-PQR-40061926,LINEA: 6013614881 NOMBRE: Pilar Moreno ...,0.0,factura llego pago pasado,0,BENEFICIO DE RETENCION,2023-29,0.92
1920,MDM-PQR-40062400,CLIENTE INDICA QUE SE LE AUMENTO LA FACTURA,0.0,aumento factura,2,BAJA NO INGRESADA,2023-29,0.90


In [69]:
retipificacion=retipificacion[['index', 'tipificacion','f1-score']]
retipificacion

,index,tipificacion,f1-score
0,2023-29,COMPENSACION,0.93
6,2023-29,COMPENSACION,0.93
7,2023-29,COMPENSACION,0.93
9,2023-29,COMPENSACION,0.93
17,2023-29,INCREMENTOS DE FACTURA,0.93
...,...,...,...
1906,2023-29,BAJA NO INGRESADA,0.90
1911,2023-29,COMPENSACION,0.93
1914,2023-29,BENEFICIO DE RETENCION,0.92
1920,2023-29,BAJA NO INGRESADA,0.90


In [71]:


retipificacion.to_csv('/content/retipificacion.csv', mode='a', index=False, header=False)
 
pd.read_csv('/content/retipificacion.csv')

# print message

#!rclone copy  /content/retipificacion.csv one:/analitica_datos/tracking_c_facturado

,index,tipificacion,f1-score
0,2023-28,BAJA NO INGRESADA,0.90
1,2023-28,BAJA NO INGRESADA,0.90
2,2023-28,BAJA NO INGRESADA,0.90
3,2023-28,BAJA NO INGRESADA,0.90
4,2023-28,BAJA NO INGRESADA,0.90
...,...,...,...
1424,2023-29,BAJA NO INGRESADA,0.90
1425,2023-29,COMPENSACION,0.93
1426,2023-29,BENEFICIO DE RETENCION,0.92
1427,2023-29,BAJA NO INGRESADA,0.90


In [72]:
!rclone copy  /content/retipificacion.csv one:/analitica_datos/tracking_c_facturado

In [ ]:
import pandas as pd
from datetime import date


from datetime import datetime

# Obtener la fecha y hora actual
current_date = datetime.now()
# Get the current date
# Specify the root directory path
root_path = '/content/PRIORIZAR/SALIDAS/'

# Export the DataFrame to a CSV file with the root path and current date in the filename
filename3 = f"{root_path}c_facturados_retipificacion_{current_date}.csv"



retipificacion.to_csv(filename3, index=False)





#pandas_c_F.to_csv('c_facturados.csv', index=False)

print("DONE")